<a href="https://colab.research.google.com/github/styopochkin23/data-analytics-portfolio/blob/main/Portfolio_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Підключаємо Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Імпорти
import pandas as pd
import numpy as np
from scipy.stats import norm


Mounted at /content/drive


In [9]:
file_path = "/content/drive/MyDrive/projects/Final Project/Portfolio2.csv"
df = pd.read_csv(file_path)

In [10]:
df.columns


Index(['date', 'country', 'device', 'continent', 'channel', 'test',
       'test_group', 'event_name', 'value'],
      dtype='object')

In [11]:
# Фіксуємо 4 метрики для A/B тесту
metrics = [
    "add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new_account"
]

df['event_name'].value_counts()


,count
event_name,
session,107210
session_start,106242
page_view,101907
user_engagement,94520
first_visit,81621
scroll,73643
view_promotion,61695
view_item,44869
session_with_orders,25892


In [12]:
metrics = [
    "add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new_account"
]

results = []

for test_id in df['test'].unique():
    test_data = df[df['test'] == test_id]

    for metric in metrics:

        ev = test_data[test_data['test_group'] == 1]
        co = test_data[test_data['test_group'] == 2]

        ev_metric = ev[ev['event_name'] == metric]['value'].sum()
        ev_session = ev[ev['event_name'] == 'session']['value'].sum()

        co_metric = co[co['event_name'] == metric]['value'].sum()
        co_session = co[co['event_name'] == 'session']['value'].sum()

        p1 = ev_metric / ev_session
        p2 = co_metric / co_session

        p_pool = (ev_metric + co_metric) / (ev_session + co_session)

        z_stat = (p1 - p2) / np.sqrt(p_pool * (1 - p_pool) * (1/ev_session + 1/co_session))

        p_value = 2 * (1 - norm.cdf(abs(z_stat)))

        significant = p_value < 0.05

        results.append([
            test_id,
            metric,
            ev_metric, ev_session, p1,
            co_metric, co_session, p2,
            z_stat,
            p_value,
            significant
        ])

columns = [
    "test_number", "metric",
    "numerator_ev", "denominator_ev", "conversion_rate_ev",
    "numerator_co", "denominator_co", "conversion_rate_co",
    "z_stat", "p_value", "significant"
]

final_df = pd.DataFrame(results, columns=columns)

final_df


/tmp/ipython-input-3102297275.py:29: RuntimeWarning: invalid value encountered in scalar divide
  z_stat = (p1 - p2) / np.sqrt(p_pool * (1 - p_pool) * (1/ev_session + 1/co_session))


,test_number,metric,numerator_ev,denominator_ev,conversion_rate_ev,numerator_co,denominator_co,conversion_rate_co,z_stat,p_value,significant
0,2,add_payment_info,2344,50637,0.046290,2409,50244,0.047946,-1.240994,0.214608,False
1,2,add_shipping_info,3480,50637,0.068724,3510,50244,0.069859,-0.709557,0.477979,False
2,2,begin_checkout,4262,50637,0.084168,4313,50244,0.085841,-0.952898,0.340642,False
3,2,new_account,0,50637,0.000000,0,50244,0.000000,NaN,NaN,False
4,1,add_payment_info,1988,45362,0.043825,2229,45193,0.049322,-3.924884,0.000087,True
5,1,add_shipping_info,3034,45362,0.066884,3221,45193,0.071272,-2.603571,0.009226,True
6,1,begin_checkout,3784,45362,0.083418,4021,45193,0.088974,-2.978783,0.002894,True
7,1,new_account,0,45362,0.000000,0,45193,0.000000,NaN,NaN,False
8,4,add_payment_info,3731,105079,0.035507,3601,105141,0.034249,1.571106,0.116158,False
9,4,add_shipping_info,5128,105079,0.048801,4956,105141,0.047137,1.785795,0.074132,False


In [13]:
final_df[['z_stat', 'p_value']] = final_df[['z_stat', 'p_value']].fillna(0)
final_df['significant'] = final_df['significant'].fillna(False)

final_df


,test_number,metric,numerator_ev,denominator_ev,conversion_rate_ev,numerator_co,denominator_co,conversion_rate_co,z_stat,p_value,significant
0,2,add_payment_info,2344,50637,0.046290,2409,50244,0.047946,-1.240994,0.214608,False
1,2,add_shipping_info,3480,50637,0.068724,3510,50244,0.069859,-0.709557,0.477979,False
2,2,begin_checkout,4262,50637,0.084168,4313,50244,0.085841,-0.952898,0.340642,False
3,2,new_account,0,50637,0.000000,0,50244,0.000000,0.000000,0.000000,False
4,1,add_payment_info,1988,45362,0.043825,2229,45193,0.049322,-3.924884,0.000087,True
5,1,add_shipping_info,3034,45362,0.066884,3221,45193,0.071272,-2.603571,0.009226,True
6,1,begin_checkout,3784,45362,0.083418,4021,45193,0.088974,-2.978783,0.002894,True
7,1,new_account,0,45362,0.000000,0,45193,0.000000,0.000000,0.000000,False
8,4,add_payment_info,3731,105079,0.035507,3601,105141,0.034249,1.571106,0.116158,False
9,4,add_shipping_info,5128,105079,0.048801,4956,105141,0.047137,1.785795,0.074132,False


In [17]:
final_df.to_csv(
    "/content/drive/MyDrive/projects/Final Project/ab_test_results_fixed.csv",
    index=False,
    sep=","
)




# A/B Test Analysis – Conversion Funnel Optimization

This project analyzes the impact of multiple A/B tests on key conversion funnel events using statistical hypothesis testing.

The goal is to:
- Evaluate whether product changes led to statistically significant improvements
- Measure conversion lift across critical user journey steps
- Support product decision-making with data-driven evidence

Key funnel events analyzed:
- add_payment_info  
- add_shipping_info  
- begin_checkout  
- new_account


### Data Source

The dataset contains aggregated session and event data split by:
- test number
- experimental group (EV)
- control group (CO)

Each test includes session counts and event occurrences required to compute conversion rates and statistical significance.

Original dataset:
[📎 https://colab.research.google.com/drive/1J_TcfI0g4hz4yQ9vwPUBSTCVa5q5H1un?usp=drive_link]  ⬅️


### A/B Testing Methodology

For each metric, the following steps were applied:

1. Calculation of conversion rates:
   - EV = event_count / session_count
   - CO = event_count / session_count

2. Statistical hypothesis testing:
   - Two-sided Z-test for proportions
   - Pooled probability used for variance estimation

3. Statistical significance:
   - Confidence level: 95%
   - Significance threshold: p-value < 0.05

4. Output metrics:
   - Z-statistic
   - P-value
   - Statistical significance flag


### Final A/B Test Results

The calculated results are stored in a separate CSV file:

📄 A/B Test Results File:
[📎 https://drive.google.com/file/d/1xthNicn9jFHRs0tg0hJvxQsoa3XY_acA/view?usp=drive_link]  

The file contains the following fields:
- test_number
- metric
- conversion_rate_ev
- conversion_rate_co
- z_stat
- p_value
- significant


### Interactive Dashboard (Tableau)

An interactive Tableau dashboard was built to visualize:
- Conversion performance across funnel steps
- Statistical significance of test results
- Comparison across multiple A/B tests

📊 Tableau Dashboard:
[📎https://public.tableau.com/app/profile/oleksandr.stopochkin/viz/Portfolio2_17649401032260/AB?publish=yes]  


## Conclusions

1. Several funnel steps demonstrated statistically significant changes (p < 0.05), indicating real behavioral impact from product experiments.

2. The strongest effects were observed on intermediate funnel steps such as:
   - add_payment_info
   - begin_checkout

3. The final step (new_account) showed no statistically significant change across tests, indicating potential downstream conversion bottlenecks.

4. The analysis confirms that:
   - Not every conversion lift is statistically reliable
   - Product decisions should rely on statistical validation rather than raw percentage changes

This project demonstrates a full-cycle product A/B testing workflow:
Data → Statistical Testing → Visualization → Business Interpretation
